In [ ]:
%pip install git+https://github.com/gbdrt/mu-ppl

In [ ]:
from mu_ppl import *

## An introduction to probabilistic programming with mu-PPL - MPRI 2.40


Probabilistic programs represent random variables.

In [ ]:
def dice() -> int:
    a = sample(RandInt(1, 6), name="a")
    b = sample(RandInt(1, 6), name="b")
    return a + b

We can run the experiment representing the random variable.

In [ ]:
dice()

The law of the random variable can be computed by an inference algorithm. 

For instance the enumeration algorithm for discrete distributions.

In [ ]:
with Enumeration():
    dist: Categorical[int] = infer(dice)

We can get a sample of the distribution.


In [ ]:
sample(dist)

We can compute the statistics of the distribution  and vizualize its mass function.

In [ ]:

s = dist.stats()
print("mean: ",s[0], "\nstandard deviation: ", s[1])
viz(dist)


Conditioning can be hard: `assume(a!=b)` reject all samples that do not satisfy the property `a!=b`

In [ ]:
def hard_dice() -> int:
    a = sample(RandInt(1, 6), name="a")
    b = sample(RandInt(1, 6), name="b")
    assume (a != b)
    return a + b

with Enumeration():
    dist: Categorical[int] = infer(hard_dice)
    print(dist.stats())
    viz(dist)

In [ ]:
from typing import Tuple
import seaborn as sns 
import matplotlib.pyplot as plt

def disk() -> Tuple[float, float]:
    x = sample(Uniform(-1, 1))
    y = sample(Uniform(-1, 1))
    d2 = x**2 + y**2
    assume (d2 < 1)
    return (x, y)

with RejectionSampling(num_samples=10000):
    dist: Empirical = infer(disk)
    x, y = zip(*dist.samples)
    sns.scatterplot(x=x, y=y)
    plt.axis("scaled")
    plt.show()


Conditioning can be soft: `observe(Gaussian(d2, 0.1), o)` condition the law given observations.

In [ ]:
def position(o: float) -> Tuple[float, float]:
    x = sample(Uniform(-1, 1))
    y = sample(Uniform(-1, 1))
    d2 = x**2 + y**2
    observe(Gaussian(d2, 0.1), o)
    return (x, y)

with ImportanceSampling(num_particles=10000):
    dist: Categorical = infer(position, 0.5)
    w = dist.probs
    x, y = list(zip(*dist.values))
    plt.scatter(x, y, c=w**0.5, cmap='Reds', s=7)
    plt.axis("scaled")
    plt.show()


In [ ]:

def success(s:int) -> int:
    n = sample(RandInt(10, 20))
    observe(Binomial(n, 0.5), s)
    return n

with ImportanceSampling(num_particles=10000):
    dist: Categorical = infer(success, 10)
    print(dist.stats())
    viz(dist)
    plt.show()

### Exercises

**Question 1** 

Describe the random variable that is modeled by this program.

In [ ]:
def coin(obs: list[int]) -> float:
    p = sample(Uniform(0, 1))
    for o in obs:
        observe(Bernoulli(p), o)
    return p 

with ImportanceSampling(num_particles=10000):
    dist: Categorical = infer(coin, [0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
    print(dist.stats())
    viz(dist)

**Question 2**

Define a probabilistic program that models the number of tosses of a fair coin before getting a Tail.

**Question 3**

Use hard conditioning to define a probabilistic program that models a fair coin using two tosses of a bias coin. 

**Question 4**

Knuth and Yao proposed an algorithm that allow to simulate perfect 6 faces dice from a fair coin.

From the initial state, we follow transitions according to the outcome of the toss of a fair coin until reaching a leave where the value is output.
The 6 leaves are numbered from 11 to 16 and corresponds to the 6 faces.

![Knuth Yao automata](./knuth-yao.jpg)

D. Knuth et A. Yao.
    *Algorithms and Complexity: New Directions and Recent Results, chapter The complexity of nonuniform random number generation*.
  Academic Press, 1976.

Implement the model and check with several inference methods that the result is a perfect dice.

**Question 5**

In 1886, Francis Galton measures the rate of regression in hereditary stature.

He used a frequency table *Number of Adult Children of various Statures born of 205 Mid-Parents of various Statures*.

He divided the data into subgroups according to the average height of the two parents. He computed the median of children stature against the median mid-parent stature and recognized a straight line.

*Galton (1886) “Regression Towards Mediocrity in Hereditary Stature,” Journal
of the Anthropological Institute of Great Britain and Ireland, 15, 246–263.*

Use the following data to reproduce Galton result.

In [ ]:
import pandas as pd
import numpy as np

raw = pd.read_csv("galton.csv")
data = raw.loc[:,["parent","child"]]

data

In [ ]:
x_obs = data["parent"]
y_obs =  data["child"]

ax=plt.axes()
ax.set_xlim(50, 80)
ax.set_ylim(50,80)
plt.scatter(x_obs, y_obs, color='red', zorder=1)

In [ ]:
from typing import Tuple

def model(data):
    # modify the following model to fit the data
    a = sample(Dirac(1), name="a")
    b = sample(Dirac(1), name="b")
    return (a, b)

with ImportanceSampling(num_particles=1000):
    dist: Categorical[Tuple[float,float]] = infer(model, data)


In [ ]:
for i in range(10):
    x = np.linspace(55, 80, 2)
    a, b = dist.sample()
    plt.plot(x, a * x + b, color='blue', alpha=0.1, zorder=0)

plt.scatter(x_obs, y_obs, color='red', zorder=1)

In [ ]:
for i in range(1000):
    x = np.linspace(0, 0.02, 2)
    a, b = dist.sample()
    plt.plot(x, a * x + b, color='blue', alpha=0.1, zorder=0)

TrueSkills™ is a skill-based ranking system developped by Microsoft for use with video game on XBox Live.[https://en.wikipedia.org/wiki/TrueSkill].

The level of each player is represented by a gaussian distribution. We assume that for any player P, *skill_P~N(100,10)*.

At each match, the skill of each player follows a gaussian distribution centered on their level and with a fixed variance. We observe the winner skill W is better than the skill of the loser L and update the random variables *skill_W* and *skill_L*.

* *perf_W~ N(skill_W, 15)*
* *perf_L~ N(skill_L, 15)*
* *perf_W>perf_L*

Implement this model for 3 players and 3 matches: 
* A wins against B
* B wins against C
* A wins against C
and check the results are what is wanted.

### Hierarchical Models

 In his book *Théorie analytique des probabilités, 1812*, Laplace compute the probability that the proportion of boys and girls registered in birth records is bigger in London than in Paris given historical data.


In [ ]:
def laplace(f1: int, g1: int, f2: int, g2: int) -> float:
    p = sample(Uniform(0, 1), name="p")
    q = sample(Uniform(0, 1), name="q")
    observe(Binomial(f1 + g1, p), g1)
    observe(Binomial(f2 + g2, q), g2)
    return q > p

# Paris    1745 - 1784
fp = 377555
gp = 393386
# Londres  1664 - 1758
fl = 698958
gl = 737629

with ImportanceSampling(num_particles=100000):
    dist: Categorical = infer(laplace, fp, gp, fl, gl)

s = dist.stats()    
print("q>p with probability ", s[0],"\nStandard deviation: ",s[1])


 In his book *Théorie analytique des probabilités, 1812*, Laplace compute the probability that the proportion of boys and girls registered in birth records is bigger in London than in Paris given historical data.
